# Webscraping - Lecture Code

## The Task:

Gather information Illinois' elected state legislators here: http://www.ilga.gov/senate/default.asp

## The Tools:

1. [Requests](http://docs.python-requests.org/en/latest/user/quickstart/)
2. [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [57]:
#THESE ARE PUBLIC PHOTOES TAGGED WITH 'LAKE_MERRITT'

from instagram.client import InstagramAPI
from random import randint
import sys
import time
from collections import OrderedDict

client_id = 'b5d78f65c2b94e74ac6363d8f2f7bcbd'
client_secret = '6e00cda85eca47b281af4aa88275c5de'
access_token = '19594423.1677ed0.d298d661a5dd4587becb8a83b3cca787'
client_ip = '73.189.135.127'

#client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri

api = InstagramAPI(client_id=client_id, client_secret=client_secret, client_ips= client_ip,access_token= access_token) 

media_with_location=[]
all_media_ids = []

media_ids,next = api.tag_recent_media(tag_name='lake_merritt', count=300)
temp,max_tag=next.split('max_tag_id=')
max_tag=str(max_tag)

position_tuple = ()

for media_id in media_ids:
	all_media_ids.append(media_id.id)
	#some media_ids don't have location assoicated with them... this maybe a hacky way of doing it, but it works
	if "location" in dir(media_id):
		#you can do it this way if that is the way you want your information, I did it using Tuples to have your data more organized and easir to access once its VERY large. 
		#position = str(media_id.id) + "," + str(media_id.location.point.latitude) + ',' + str(media_id.location.point.longitude) + ';' + str(media_id.get_standard_resolution_url())

		# i found that some of the data doesn't have a full location ... so this just to check for that case. 
		if ("point" in dir(media_id.location)) and ("latitude" in dir(media_id.location.point)) and ("longitude" in dir(media_id.location.point)):
			position_tuple=(str(media_id.id),str(media_id.location.point.latitude),str(media_id.location.point.longitude),str(media_id.get_standard_resolution_url()))
			media_with_location.append(position_tuple)

counter = 1
while next and counter < 3 :
	more_media, next = api.tag_recent_media(tag_name='lake_merritt', max_tag_id=max_tag)
	temp,max_tag=next.split('max_tag_id=')
	max_tag=str(max_tag)
	for media_id in more_media:
		all_media_ids.append(media_id.id)
		#some media_ids don't have location assoicated with them... this maybe a hacky way of doing it, but it works
		if "location" in dir(media_id):
			#you can do it this way if that is the way you want your information, I did it using a "list of tuples" to have your data more organized and easir to access once its VERY large. 
			#position = str(media_id.id) + "," + str(media_id.location.point.latitude) + ',' + str(media_id.location.point.longitude) + ';' + str(media_id.get_standard_resolution_url())

			# i found that some of the data doesn't have a full location ... so this just to check for that case. 
			if ("point" in dir(media_id.location)) and ("latitude" in dir(media_id.location.point)) and ("longitude" in dir(media_id.location.point)):
				position_tuple=(str(media_id.id),str(media_id.location.point.latitude),str(media_id.location.point.longitude),str(media_id.get_standard_resolution_url()))
				media_with_location.append(position_tuple)
	counter+=1



#delete duplicates just incase...
media_with_location=list(OrderedDict.fromkeys(media_with_location))
all_media_ids=list(OrderedDict.fromkeys(all_media_ids))

#check length of all media ids to make sure pagination is working
print "Length of All Media IDS:", len(all_media_ids)

#some prints to validate your information
print "media_with_location"
print media_with_location
print len(media_with_location)

#this is how to access a member within the list
print media_with_location[0][2]

Length of All Media IDS: 73
media_with_location
[('1022203687264871862_913245280', '37.782580261', '-122.21532139', 'https://scontent.cdninstagram.com/hphotos-xaf1/t50.2886-16/11723892_908448552564991_636535954_n.mp4'), ('920625456882307349_38347483', '37.804414291', '-122.251496774', 'https://scontent.cdninstagram.com/hphotos-xpa1/t51.2885-15/e15/924756_412596178887551_2004166462_n.jpg'), ('877240390495123453_242151420', '37.80737242', '-122.254306971', 'https://scontent.cdninstagram.com/hphotos-xpa1/t51.2885-15/e15/927738_751654308260433_880924362_n.jpg'), ('864490978064204376_10985767', '37.80688791', '-122.258809658', 'https://scontent.cdninstagram.com/hphotos-xpa1/t51.2885-15/e15/924055_1523219964607110_1900805353_n.jpg'), ('817814980139470520_49708378', '37.799116119', '-122.258911085', 'https://scontent.cdninstagram.com/hphotos-xpf1/t51.2885-15/e15/10522790_448992538577149_1778692219_n.jpg'), ('742862633002919198_481965824', '37.805536344', '-122.254301678', 'https://scontent.cd

In [59]:
import csv

data=media_with_location

with open('lake_merritt.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['instaID','lat','long','URL'])
    for row in data:
        csv_out.writerow(row)

## Step 1: Make a Get Request and Read in HTML

We use `requests` library to:
1. make a GET request to the page
2. read in the html of the page

In [53]:
import requests
from bs4 import BeautifulSoup
#these are instagrams -both public and private - from within 1mile of lake merritt
# make a GET request for about 1mile - about 1600m around lake merritt, used geocode in r to get long and lat of lake merritt
req = requests.get('https://api.instagram.com/v1/media/search?lat=37.80124&lng=-122.2583&distance=1600&access_token=19594423.1677ed0.d298d661a5dd4587becb8a83b3cca787')
# read the content of the server’s response
src = req.text

In [54]:
soup = BeautifulSoup(src)
# take a look
print soup.prettify()

<html>
 <body>
  <p>
   {"meta":{"code":200},"data":[{"attribution":null,"tags":[],"type":"image","location":{"latitude":37.8042488,"name":"VENUE Oakland","longitude":-122.2703018,"id":123672253},"comments":{"count":3,"data":[{"created_time":"1448344993","text":"Back game proper","from":{"username":"reinadelacastles","profile_picture":"https:\/\/scontent.cdninstagram.com\/hphotos-xfp1\/t51.2885-19\/s150x150\/12231085_192408207766124_1953862817_a.jpg","id":"5919124","full_name":""},"id":"1125121811032961800"},{"created_time":"1448345103","text":"Tatt game hard af","from":{"username":"funsizeniecey","profile_picture":"https:\/\/scontent.cdninstagram.com\/hphotos-xfa1\/t51.2885-19\/s150x150\/11950676_753314974779100_929426059_a.jpg","id":"400126054","full_name":"\ud83d\udc52FB : ShaniceMitche"},"id":"1125122737512772398"},{"created_time":"1448345659","text":"Ain't been there since my 25th birthday forgot about that spot","from":{"username":"dezzydez88","profile_picture":"https:\/\/sconten

In [55]:
len(soup.prettify())

64893

## Step 2: Soup it

Now we use the `BeautifulSoup` function to parse the reponse into an HTML tree. This returns an object (called a **soup object**) which contains all of the HTML in the original document.

In [4]:
# parse the response into an HTML tree
soup = BeautifulSoup(src)
# take a look
print soup.prettify()[:1000]

<html lang="en">
 <head>
  <title>
   Illinois General Assembly - Senate Members
  </title>
  <link href="/style/lis.css" rel="stylesheet" type="text/css"/>
  <link href="/style/print.css" media="print" rel="stylesheet" type="text/css"/>
  <link href="http://info.er.usgs.gov/public/gils/gilsexec.html" rel="GILS"/>
  <link href="/LISlogo1.ico" rel="Shortcut Icon"/>
  <script language="JavaScript" type="text/javascript">
   <!--

if(window.event + "" == "undefined") event = null;
function HM_f_PopUp(){return false};
function HM_f_PopDown(){return false};
popUp = HM_f_PopUp;
popDown = HM_f_PopDown;

//-->
  </script>
  <!--
    option explicit
  -->
  <meta content='(PICS-1.1 "http://www.weburbia.com/safe/ratings.htm" l r (s 0))' http-equiv="PICS-Label"/>
  <meta content="Government" name="classification"/>
  <meta content="Global" name="distribution"/>
  <meta content="General" name="rating"/>
  <meta content="IL" name="contactState"/>
  <meta content="Illinois General Assembly" name="


## Step 3: Find Elements

BeautifulSoup has a number of functions to find things on a page. Like other webscraping tools, Beautiful Soup lets you find elements by their:

1. HTML tags
2. HTML Attributes
3. CSS Selectors


Let's search first for **HTML tags**. 

The function `find_all` searches the `soup` tree to find all the elements with an a particular HTML tag, and returns all of those elements.

What does the example below do?

In [5]:
# find all elements in a certain tag
# these two lines of code are equivilant

soup.find_all("a")

[<a href="/default.asp"><img alt="Illinois General Assembly" border="0" height="49" src="/images/logo_sm.gif" width="462"/></a>,
 <a class="mainmenu" href="/">Home</a>,
 <a class="mainmenu" href="/legislation/" onblur="HM_f_PopDown('elMenu1')" onfocus="HM_f_PopUp('elMenu1',event)" onmouseout="HM_f_PopDown('elMenu1')" onmouseover="HM_f_PopUp('elMenu1',event)">Legislation &amp; Laws</a>,
 <a class="mainmenu" href="/senate/" onblur="HM_f_PopDown('elMenu3')" onfocus="HM_f_PopUp('elMenu3',event)" onmouseout="HM_f_PopDown('elMenu3')" onmouseover="HM_f_PopUp('elMenu3',event)">Senate</a>,
 <a class="mainmenu" href="/house/" onblur="HM_f_PopDown('elMenu2')" onfocus="HM_f_PopUp('elMenu2',event)" onmouseout="HM_f_PopDown('elMenu2')" onmouseover="HM_f_PopUp('elMenu2',event)">House</a>,
 <a class="mainmenu" href="/mylegislation/" onblur="HM_f_PopDown('elMenu4')" onfocus="HM_f_PopUp('elMenu4',event)" onmouseout="HM_f_PopDown('elMenu4')" onmouseover="HM_f_PopUp('elMenu4',event)">My Legislation</a>,
 

**NB**: Because `find_all()` is the most popular method in the Beautiful Soup search API, you can use a shortcut for it. If you treat the BeautifulSoup object as though it were a function, then it’s the same as calling `find_all()` on that object. These two lines of code are equivalent:

In [7]:
soup.find_all("a")
soup("a")

[<a href="/default.asp"><img alt="Illinois General Assembly" border="0" height="49" src="/images/logo_sm.gif" width="462"/></a>,
 <a class="mainmenu" href="/">Home</a>,
 <a class="mainmenu" href="/legislation/" onblur="HM_f_PopDown('elMenu1')" onfocus="HM_f_PopUp('elMenu1',event)" onmouseout="HM_f_PopDown('elMenu1')" onmouseover="HM_f_PopUp('elMenu1',event)">Legislation &amp; Laws</a>,
 <a class="mainmenu" href="/senate/" onblur="HM_f_PopDown('elMenu3')" onfocus="HM_f_PopUp('elMenu3',event)" onmouseout="HM_f_PopDown('elMenu3')" onmouseover="HM_f_PopUp('elMenu3',event)">Senate</a>,
 <a class="mainmenu" href="/house/" onblur="HM_f_PopDown('elMenu2')" onfocus="HM_f_PopUp('elMenu2',event)" onmouseout="HM_f_PopDown('elMenu2')" onmouseover="HM_f_PopUp('elMenu2',event)">House</a>,
 <a class="mainmenu" href="/mylegislation/" onblur="HM_f_PopDown('elMenu4')" onfocus="HM_f_PopUp('elMenu4',event)" onmouseout="HM_f_PopDown('elMenu4')" onmouseover="HM_f_PopUp('elMenu4',event)">My Legislation</a>,
 

That's a lot! Many elements on a page will have the same html tag. For instance, if you search for everything with the `a` tag, you're likely to get a lot of stuff, much of which you don't want. What if we wanted to search for HTML tags ONLY with certain attributes, like particular CSS classes? 

We can do this by adding an additional argument to the `find_all`

In the example below, we are finding all the `a` tags, and then filtering those with `class = "sidemenu"`.

In [8]:
# Get only the 'a' tags in 'sidemenu' class
soup("a", class_="sidemenu")

[<a class="sidemenu" href="/senate/default.asp">  Members  </a>,
 <a class="sidemenu" href="/senate/committees/default.asp">  Committees  </a>,
 <a class="sidemenu" href="/senate/schedules/default.asp">  Schedules  </a>,
 <a class="sidemenu" href="/senate/journals/default.asp">  Journals  </a>,
 <a class="sidemenu" href="/senate/transcripts/default.asp">  Transcripts  </a>,
 <a class="sidemenu" href="/senate/rules.asp">  Rules  </a>,
 <a class="sidemenu" href="/senate/audvid.asp">  Live Audio/Video  </a>]

Oftentimes a more efficient way to search and find things on a website is by **CSS selector.** For this we have to use a different method, `select()`. Just pass a string into the `.select()` to get all elements with that string as a valid CSS selector.

In the example above, we can use "a.sidemenu" as a CSS selector, which returns all `a` tags with class `sidemenu`.

In [10]:
# get elements with "a.sidemenu" CSS Selector.
# shortcut
soup.select("a.sidemenu")
soup.select(".sidemenu a")

[<a class="sidemenu" href="/senate/default.asp">  Members  </a>,
 <a class="sidemenu" href="/senate/committees/default.asp">  Committees  </a>,
 <a class="sidemenu" href="/senate/schedules/default.asp">  Schedules  </a>,
 <a class="sidemenu" href="/senate/journals/default.asp">  Journals  </a>,
 <a class="sidemenu" href="/senate/transcripts/default.asp">  Transcripts  </a>,
 <a class="sidemenu" href="/senate/rules.asp">  Rules  </a>,
 <a class="sidemenu" href="/senate/audvid.asp">  Live Audio/Video  </a>]

In [14]:
# this is a list
soup.select("a.sidemenu")

# we first want to get an individual tag object
first_link = soup.select("a.sidemenu")[0]

# check out its class
type(first_link)
print(first_link)

<a class="sidemenu" href="/senate/default.asp">  Members  </a>


It's a tag! Which means it has a `text` member:

In [12]:
print first_link.text

  Members  


Sometimes we want the value of certain attributes. This is particularly relevant for `a` tags, or links, where the `href` attribute tells us where the link goes.

You can access a tag’s attributes by treating the tag like a dictionary:

In [15]:
#URL that corresponds to the first link in the menu
print first_link['href']

/senate/default.asp


In [16]:
# Get just the href (url) attribute from the first 10 links
for link in soup('a')[:10]:
    print link['href']

/default.asp
/
/legislation/
/senate/
/house/
/mylegislation/
/sitemap.asp
/senate/default.asp
/senate/committees/default.asp
/senate/schedules/default.asp


## Let's Do This.

Believe it or not, that's all you need to scrape a website. Let's apply these skills to scrape http://www.ilga.gov/senate/default.asp

First, make the get request

In [17]:
# make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp')
# read the content of the server’s response
src = req.text
# soup it
soup = BeautifulSoup(src)

Now let's try to get a list of rows in that table. Rows are identified by the `tr` tag.

In [18]:
# get all tr elements
rows = soup.find_all("tr")
len(rows)

81

But remember, `find_all` gets all the elements with the `tr` tag. We can use smart CSS selectors to get only the rows we want.

In [23]:
# returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')
print rows[5].prettify()

<tr>
 <td bgcolor="EBEBEB" class="detail" width="40%">
  <a href="/senate/Senator.asp?GA=99&amp;MemberID=2276">
   Scott M. Bennett
  </a>
 </td>
 <td align="center" bgcolor="EBEBEB" class="detail" width="15%">
  <a href="SenatorBills.asp?MemberID=2276">
   Bills
  </a>
 </td>
 <td align="center" bgcolor="EBEBEB" class="detail" width="15%">
  <a href="SenCommittees.asp?MemberID=2276">
   Committees
  </a>
 </td>
 <td align="center" bgcolor="EBEBEB" class="detail" width="15%">
  52
 </td>
 <td align="center" bgcolor="EBEBEB" class="detail" width="15%">
  D
 </td>
</tr>



We can use the `select` method on anything. Let's say we want to find everything with the CSS selector `td.detail` in an item of the list we created above.

In [25]:
# returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')

# Look at just the third item
row = rows[4]

# select only those 'td' tags with class 'detail'
detailCells = row.select('td.detail')

detailCells

[<td bgcolor="white" class="detail" width="40%"><a href="/senate/Senator.asp?GA=99&amp;MemberID=2208">Jason A. Barickman</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?MemberID=2208">Bills</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?MemberID=2208">Committees</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%">53</td>,
 <td align="center" bgcolor="white" class="detail" width="15%">R</td>]

Most of the time, we're interested in the actual **text** of a website, not its tags. Remember, to get the text of an HTML element, use the `text` member.

In [28]:
# returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')

# Look at just the third item
row = rows[2]

# select only those 'td' tags with class 'detail'
detailCells = row.select('td.detail')

# Keep only the text in each of those cells
rowData = [cell.text for cell in detailCells]

print rowData

[u'Pamela J. Althoff', u'Bills', u'Committees', u'32', u'R']


Now we can combine the beautifulsoup tools with our basic python skills to scrape an entire web page.

In [38]:
# returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')

# Look at just the third item
row = rows[2]

# select only those 'td' tags with class 'detail'
detailCells = row.select('td.detail')

# Keep only the text in each of those cells
rowData = [cell.text for cell in detailCells]

# check em out
print rowData[0] # Name
print rowData[3] # district
print rowData[4] # party

print detailCells[2].a['href']

Pamela J. Althoff
32
R
SenCommittees.asp?MemberID=2130


Let's use a for loop to get 'em all!

In [40]:
# Create empty list to store our data
members = []

# returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')

# loop through all rows
for row in rows:
    # select only those 'td' tags with class 'detail'
    detailCells = row.select('td.detail')
    
    # get rid of junk rows, skip rows that don't contain the five elements
    if len(detailCells) is not 5: 
        continue
        
    # Keep only the text in each of those cells
    rowData = [cell.text for cell in detailCells]
    
    # Collect information
    name = rowData[0]
    link = detailCells[2].a['href']
    district = int(rowData[3])
    party = rowData[4]
    
    # Store in a tuple
    tup = (name,district,party,link)
    
    # Append to list
    members.append(tup)
    
members

[(u'Pamela J. Althoff', 32, u'R', 'SenCommittees.asp?MemberID=2130'),
 (u'Neil Anderson', 36, u'R', 'SenCommittees.asp?MemberID=2275'),
 (u'Jason A. Barickman', 53, u'R', 'SenCommittees.asp?MemberID=2208'),
 (u'Scott M. Bennett', 52, u'D', 'SenCommittees.asp?MemberID=2276'),
 (u'Jennifer Bertino-Tarrant', 49, u'D', 'SenCommittees.asp?MemberID=2210'),
 (u'Daniel Biss', 9, u'D', 'SenCommittees.asp?MemberID=2209'),
 (u'Tim Bivins', 45, u'R', 'SenCommittees.asp?MemberID=2160'),
 (u'William E. Brady', 44, u'R', 'SenCommittees.asp?MemberID=2108'),
 (u'Melinda Bush', 31, u'D', 'SenCommittees.asp?MemberID=2211'),
 (u'James F. Clayborne, Jr.', 57, u'D', 'SenCommittees.asp?MemberID=2097'),
 (u'Jacqueline Y. Collins', 16, u'D', 'SenCommittees.asp?MemberID=2123'),
 (u'Michael Connelly', 21, u'R', 'SenCommittees.asp?MemberID=2212'),
 (u'John J. Cullerton', 6, u'D', 'SenCommittees.asp?MemberID=2098'),
 (u'Thomas Cullerton', 23, u'D', 'SenCommittees.asp?MemberID=2213'),
 (u'Bill Cunningham', 18, u'D'

### Challenge 1

Make a function that accepts a URL, scrapes the URL for its senators, and returns a list of tuples containing information about each senator. 

In [46]:
def get_members(url):
    # make a GET request
    req = requests.get(url)
    # read the content of the server’s response
    src = req.text
    # soup it
    soup = BeautifulSoup(src)
    members = []

    # returns every ‘tr tr tr’ css selector in the page
    rows = soup.select('tr tr tr')

    # loop through all rows
    for row in rows:
        # select only those 'td' tags with class 'detail'
        detailCells = row.select('td.detail')

        # get rid of junk rows, skip rows that don't contain the five elements
        if len(detailCells) is not 5: 
            continue

        # Keep only the text in each of those cells
        rowData = [cell.text for cell in detailCells]

        # Collect information
        name = rowData[0]
        link = detailCells[2].a['href']
        district = int(rowData[3])
        party = rowData[4]
        # Store in a tuple
        tup = (name,district,party,link)

        # Append to list
        members.append(tup)

    return members  

In [47]:
get_members('http://www.ilga.gov/senate/default.asp')[1]

(u'Neil Anderson', 36, u'R', 'SenCommittees.asp?MemberID=2275')

### Challenge 2

Given the `senateMembers`  list, create a new dictionary `members_dict` which has as its keys the district number (e.g. ` 32`) and as its values, the entire tuple as returned by `get_members()` (name, district number, party, url). We can do this because the district number is a unique identifier for each senator.

Calling `members_dict[32]`, for example, should return the 4-tuple:

```(u'Pamela J. Althoff',
32,
u'R',
'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=1911&Primary=True')
```.

In [155]:
# YOUR CODE HERE